# Setup

In [ ]:
%pip install pandas transformers datasets torch scikit-learn evaluate seaborn imblearn accelerate>=0.26.0

# Preparação de dados
Carrega o dataset a ser utilizado para fine-tuning e seleciona os atributos mais relevantes.

In [ ]:
import pandas as pd
import seaborn as sns
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

Faz o download do dataset anotado no diretório ./data

In [ ]:
import os

if not os.path.exists('./data/covidbr_labeled.csv'):
  !mkdir data
  !curl -L -o ./data/covidbr_labeled.csv https://zenodo.org/records/5193932/files/covidbr_labeled.csv
else:
    print("File already exists. Skipping download.")

In [ ]:
original_dataset_df = pd.read_csv('./data/covidbr_labeled.csv')
original_dataset_df

In [ ]:
dataset_df = original_dataset_df[["text", "misinformation"]]
dataset_df

# Análise exploratória de dados

O objetivo é entender melhor e sumarizar as características dos dados, analisando quantidade e tipos de atributos, verificando distribuição do atributo alvo, identificando padrões e anomalias, removendo atributos que pareçam irrelevantes ou problemáticos, etc. Utilize gráficos e sumarizações estatísticas para a EDA. Verifique potenciais problemas nos dados, como por exemplo, a necessidade de normalizar os atributos, balancear classes, ou remover instâncias ou atributos por inconsistências nos dados.

- P1. Qual a quantidade e tipos de atributos? Existem inconsistências?
  - Quais são os atributos disponíveis?
  - Existem inconsistências nos atributos? (Atributos vazios, potenciais erros, etc)
  - Existem atributos que necessitam ser removidos ou transformados?
- P2. Qual a distribuição do atributo alvo?
  - Quais são as classes alvo? Qual a distribuição entre as classes? Está balanceada ou desbalanceada?
- P3. Quais os padrões e anomalias dos atributos?



## P1. Qual a quantidade e tipos de atributos? Existem inconsistências?

In [ ]:
dataset_df.info(verbose = False, memory_usage = False, show_counts = True) # mostra o tipo e a quantidade de itens não nulos de cada coluna

In [ ]:
dataset_df.dtypes

## P2. Qual a distribuição do atributo alvo?

In [ ]:
dataset_df['misinformation'].describe(include='all')

# Pré-processamento

## Tokenização

Carrega o tokenizador para `bert-base-portuguese-cased` (BERTimbau)

In [ ]:
from transformers import AutoTokenizer  # Or BertTokenizer

tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)

Aplica a tokenização para todas as instâncias de `text`

In [ ]:
def tokenize_function(examples):
    print(examples)
    return tokenizer(str(examples), padding="max_length", truncation=True, max_length=128)

# Apply the tokenizer to the dataset
tokenized_datasets = dataset_df.apply(lambda row: tokenize_function(row["text"]), axis=1)

# Inspect tokenized samples
tokenized_df = pd.DataFrame(tokenized_datasets, columns=["tk_text"])
tokenized_df

In [ ]:
data = pd.concat([tokenized_df, dataset_df["misinformation"]], axis=1, join="inner")
data

## Balanceamento de classes

Utilizando o cálculo de class_weights.

Fonte: https://medium.com/@heyamit10/fine-tuning-bert-for-classification-a-practical-guide-b8c1c56f252c

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

labels = data["misinformation"]
class_weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)
print(class_weights)

# Fine-tuning

## Configuração do modelo

In [ ]:
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
from transformers import BertModel
from transformers import BertForSequenceClassification

model_name = 'neuralmind/bert-base-portuguese-cased'
model = BertForSequenceClassification.from_pretrained(model_name)

print(model.config)

In [ ]:
# Freeze all layers except the classifier
for param in model.bert.parameters():
    param.requires_grad = False

# Keep only the classification head trainable
for param in model.classifier.parameters():
    param.requires_grad = True

print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

## Configuração do treinamento

In [ ]:
from transformers import TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",           # Directory for saving model checkpoints
    #evaluation_strategy="epoch",     # Evaluate at the end of each epoch
    learning_rate=5e-5,              # Start with a small learning rate
    per_device_train_batch_size=16,  # Batch size per GPU
    per_device_eval_batch_size=16,
    num_train_epochs=3,              # Number of epochs
    weight_decay=0.01,               # Regularization
    save_total_limit=2,              # Limit checkpoints to save space
    #load_best_model_at_end=True,     # Automatically load the best checkpoint
    logging_dir="./logs",            # Directory for logs
    logging_steps=100,               # Log every 100 steps
    fp16=True                        # Enable mixed precision for faster training
)

print(training_args)

## 5-Fold Cross Validation

In [ ]:
from transformers import Trainer
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold

fold_results = []

# prepare cross validation
k=5
kf = KFold(n_splits=k, random_state=1, shuffle=True)

for fold, (train_index, val_index) in enumerate(kf.split(data)):
    print(f"Fold {fold + 1}")
    
    # Split the data into training and validation sets
    train_df = data.iloc[train_index]
    val_df = data.iloc[val_index]
    
    # Convert to Hugging Face Dataset format
    train_dataset = train_df.to_dict(orient="list")
    val_dataset = val_df.to_dict(orient="list")
    
    # Create a Trainer instance
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=lambda p: {"f1": f1_score.compute(predictions=p.predictions.argmax(-1), references=p.label_ids)["f1"]}
    )
    
    # Train the model
    trainer.train()
    
    # Evaluate the model
    eval_result = trainer.evaluate()
    print(f"Fold {fold + 1} F1 Score: {eval_result['eval_f1']}")
    fold_results.append(eval_result['eval_f1'])

# Calculate and print the mean F1 score
mean_f1 = sum(fold_results) / len(fold_results)
print(f"Mean F1 Score: {mean_f1}")

ref https://www.philschmid.de/k-fold-as-cross-validation-with-a-bert-text-classification-example